# Introduction #

In the last lesson, we learned how to use the transfer learning technique called fine tuning to improve our classifier. The basis of this technique was making use of the hierarchy of features within the base.

In this lesson, we'll make use another property: **invariance**. We'll see how we can extend the translation invariance we learned about in Lesson 3 to other kinds of invariance which will improve our classifier even further.

# Understanding Invariance #

An idea we touched on when looking at pooling was **invariance**. Broadly, to say that a machine learning model is invariant to some property means that it will treat two examples differing only by that property as being exactly the same. We said that pooling made a network invariant to small shifts in the position of a feature. The network simply ignores that kind of difference.

To solve a classification problem means to find a model that is invariant with respect to the class labels: it should assign the same label to all images in the same class.

Giving our model translation invariance is helpful because things in the same class tend to have the same features, even if those features are shifted around. If our classifier detects a beak and feathers, our picture is almost certainly of a bird and not a fish or a horse, regardless of where those beak and features happen to be.

# Learning Invariance #

So one way to improve a model is to build an invariance into the model itself, like we did with pooling. The other way is to *teach* the model the invariance. This is what you are doing when you show the model many different images with the same class label. It learns to treat those images the same.

<!--TODO: images of the same class-->
<figure>
<img src="" width=400 alt="Images of the same class.">
</figure>

A typical dataset, however, won't have enough examples to fully teach the model all the ways in which images from a class might be different. If you need to improve the accuracy of your model, the best thing is to collect more data. Another (much less expensive) way is to *modify* the data you already have.

The idea behind **data augmentation** is that you can teach your model an invariance by transforming your existing data in ways that images of a class in unseen data might vary. For instance, if you are classifying cars, your classifier should know that whether a car is facing left-to-right or right-to-left doesn't affect what class the car is in. If a car is a Honda in one direction, it's also a Honda in any other direction.

<!--TODO: cars in different directions -->
<figure>
<img src="" width=400 alt="Same class of car in different directions.">
</figure>

So, if you augment your dataset by flipping all of your images, you help your classifier to learn that this is a distinction it should ignore.

<!--TODO: data for free-->
<figure>
<img src="" width=400 alt="Augmented cars.">
</figure>

TensorFlow includes a number of data augmentations in its `images` module, and there are a number of others in the **TensorFlow Addons** library.

# Example - Training with Data Augmentation #

Let's see how we can use the data augmentations provided by Keras to improve the performance of the classifier from Lesson 1.

## Step 1 - Create Data Loader with Augmentations ##

Define augmentations.

In [ ]:
import visiontools

augment = visiontools.make_augmentor(
    horizontal_flip=True,
    brightness_delta=0.2,
    hue_delta=0.5,
    saturation_range=[0.0, 1.25],
    contrast_range=[0.8, 1.5],
)

In [ ]:
#$HIDE_INPUT$
# Imports
import os
import warnings
import numpy as np
import visiontools
from visiontools import StanfordCars
import tensorflow as tf
import tensorflow_datasets as tfds

# Reproducibility
def seed_everything(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 31415
seed_everything(seed)
warnings.filterwarnings("ignore")

# Load training and validation sets
DATA_DIR = '/kaggle/input/stanford-cars-for-learn/'
(ds_train_, ds_valid_), ds_info = tfds.load(
    'stanford_cars/simple',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    data_dir=DATA_DIR,
)
print(("Loaded {} training examples " +
       "and {} validation examples " +
       "with classes {}.").format(
           ds_info.splits['train'].num_examples,
           ds_info.splits['test'].num_examples,
           ds_info.features['label'].names))

# Create data pipeline
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE
SIZE = [192, 192]
preprocess = visiontools.make_preprocessor(size=SIZE)

ds_train = (ds_train_
            .map(preprocess)
            .cache()
            .shuffle(ds_info.splits['train'].num_examples)
            .map(augment, AUTO)            
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid = (ds_valid_
            .map(preprocess)
            .cache()
            .shuffle(ds_info.splits['test'].num_examples)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

Let's take a look at what effect these augmentations can have.

In [ ]:
plt.figure(figsize=(15, 7))
for i, (image, label) in enumerate(ds_train.unbatch().take(18)):
    plt.subplot(3, 6, i+1)
    plt.axis('off')
    plt.imshow(image)
plt.tight_layout()
plt.show()

## Step 2 - Define Model ##

We'll continue with the VGG16 model we've used throughout this microcourse.

In [ ]:
from tensorflow.keras import Sequential
import tensorflow.keras.layers as layers

pretrained_base = tf.keras.models.load_model(
    '/kaggle/input/cv-course-models/cv-course-models/vgg16-pretrained-base',
)
pretrained_base.trainable = False

model = Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

## Step 3 - Train and Evaluate ##

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=15
)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

# Conclusion #

The `albumentations` library is another source for augmentations; it has been popular with Kagglers in competitions.